In [1]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials
from array import array
import os
from PIL import Image
import sys
import time

from dotenv import load_dotenv
import os

# Încarcă variabilele de mediu din fișierul .env
load_dotenv()
'''
Authenticate
Authenticates your credentials and creates a client.
'''
subscription_key = os.environ["VISION_KEY"]
endpoint = os.environ["VISION_ENDPOINT"]
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))
'''
END - Authenticate
'''

def returnAllWordsThatAppearInImage(imagePath):
    """ 
    Returns a list of words that are detected in an image given by its path
    """
    img = open(imagePath, "rb")
    read_response = computervision_client.read_in_stream(
        image=img,
        mode="Printed",
        raw=True
    )
    operation_id = read_response.headers['Operation-Location'].split('/')[-1]
    while True:
        read_result = computervision_client.get_read_result(operation_id)
        if read_result.status not in ['notStarted', 'running']:
            break
        time.sleep(1)

    
    words = []
    if read_result.status == OperationStatusCodes.succeeded:
        for text_result in read_result.analyze_result.read_results:
            for line in text_result.lines:
                for word in line.text.split():
                    words.append(word)
    return words


def returnAllCharactersThatAppearInAListOfWords(words):
    """  
    Returns all characters that appear in a list of words
    words: list of words
    """
    characters = []
    for word in words:
        for character in word:
            characters.append(character)
    return characters

In [2]:
from Levenshtein import distance as LD
from scipy.spatial.distance import hamming as HD

def distantaLevenshtein(str1, str2):
    # str1 is empty
    if len(str1) == 0:
        return len(str2)
    # str2 is empty
    if len(str2) == 0:
        return len(str1)
    if str1[-1] == str2[-1]:
        return distantaLevenshtein(str1[0:len(str1)-1], str2[0:len(str2)-1])
    return 1 + min(
          # Insert     
        distantaLevenshtein(str1, str2[0:len(str2)-1]),
        min(
              # Remove
            distantaLevenshtein(str1[0:len(str1)-1], str2),
          # Replace
            distantaLevenshtein(str1[0:len(str1)-1], str2[0:len(str2)-1])
        )    
    )

def calitateaProcesuluiDeRecunoastereATextuluiCuDistantaLevenshtein(imagePath, normalWords, normalCharacters):
    """ 
    Masoara calitatea procesului de recunoastere a cuvintelor, respectiv caracterelor folosind distanta Levenshtein
    normalWords - cuvintele din imagine
    normalCharacters - literele din imagine
    """
    recognisedWords = returnAllWordsThatAppearInImage(imagePath)
    recognisedCharacters = returnAllCharactersThatAppearInAListOfWords(recognisedWords)

    calitateaCuvintelor = distantaLevenshtein(recognisedWords, normalWords)
    calitateaCaracterelor = distantaLevenshtein(recognisedCharacters, normalCharacters)

    return calitateaCuvintelor, calitateaCaracterelor

def calitateaProcesuluiDeRecunoastereATextuluiCuDistantaLevenshtein2(imagePath, normalWords, normalCharacters):
    """ 
    Masoara calitatea procesului de recunoastere a cuvintelor, respectiv caracterelor folosind distanta Levenshtein
    normalWords - cuvintele din imagine
    normalCharacters - literele din imagine
    """
    recognisedWords = returnAllWordsThatAppearInImage(imagePath)
    recognisedCharacters = returnAllCharactersThatAppearInAListOfWords(recognisedWords)

    calitateaCuvintelor = LD(recognisedWords, normalWords)
    calitateaCaracterelor = LD(recognisedCharacters, normalCharacters)

    return calitateaCuvintelor, calitateaCaracterelor



def hamming_distance(string1: str, string2: str) -> int:
    """Return the Hamming distance between two strings."""
    if len(string1) != len(string2):
        raise ValueError("Strings must be of equal length.")
    dist_counter = 0
    for n in range(len(string1)):
        if string1[n] != string2[n]:
            dist_counter += 1
    return dist_counter

def calitateaProcesuluiDeRecunoastereATextuluiCuDistantaHamming(imagePath, normalWords, normalCharacters):
    """ 
    Masoara calitatea procesului de recunoastere a cuvintelor, respectiv caracterelor folosind distanta Hamming
    normalWords - cuvintele din imagine
    normalCharacters - literele din imagine
    """
    recognisedWords = returnAllWordsThatAppearInImage(imagePath)
    recognisedCharacters = returnAllCharactersThatAppearInAListOfWords(recognisedWords)

    calitateaCuvintelor = hamming_distance(recognisedWords, normalWords)
    calitateaCaracterelor = hamming_distance(recognisedCharacters, normalCharacters)

    return calitateaCuvintelor, calitateaCaracterelor

def calitateaProcesuluiDeRecunoastereATextuluiCuDistantaHamming2(imagePath, normalWords, normalCharacters):
    """ 
    Masoara calitatea procesului de recunoastere a cuvintelor, respectiv caracterelor folosind distanta Hamming
    normalWords - cuvintele din imagine
    normalCharacters - literele din imagine
    """
    recognisedWords = returnAllWordsThatAppearInImage(imagePath)
    recognisedCharacters = returnAllCharactersThatAppearInAListOfWords(recognisedWords)

    calitateaCuvintelor = HD(recognisedWords, normalWords)
    calitateaCaracterelor = HD(recognisedCharacters, normalCharacters)

    return calitateaCuvintelor, calitateaCaracterelor


def longestCommonSubsequence(X, Y): 
    if len(X) == 0 or len(Y) == 0: 
        return 0
    elif X[-1] == Y[-1]: 
        return 1 + longestCommonSubsequence(X[0: len(X)-1], Y[0:len(Y)-1]) 
    else: 
        return max(longestCommonSubsequence(X, Y[0: len(Y)-1]), longestCommonSubsequence(X[0:len(X)-1], Y)) 
    
def calitateaProcesuluiDeRecunoastereATextuluiCuLongestCommonSubsequence(imagePath, normalWords, normalCharacters):
    """ 
    Masoara calitatea procesului de recunoastere a cuvintelor, respectiv caracterelor folosind Longest Common Subsequence
    normalWords - cuvintele din imagine
    normalCharacters - literele din imagine
    """
    recognisedWords = returnAllWordsThatAppearInImage(imagePath)
    recognisedCharacters = returnAllCharactersThatAppearInAListOfWords(recognisedWords)

    calitateaCuvintelor = longestCommonSubsequence(recognisedWords, normalWords)
    calitateaCaracterelor = longestCommonSubsequence(recognisedCharacters, normalCharacters)

    return calitateaCuvintelor, calitateaCaracterelor


def jaro_distance(s1, s2) :
 
    # If the strings are equal 
    if (s1 == s2) :
        return 1.0; 
 
    # Length of two strings 
    len1 = len(s1);
    len2 = len(s2); 
 
    if (len1 == 0 or len2 == 0) :
        return 0.0; 
 
    # Maximum distance upto which matching 
    # is allowed 
    max_dist = (max(len(s1), len(s2)) // 2 ) - 1; 
 
    # Count of matches 
    match = 0; 
 
    # Hash for matches 
    hash_s1 = [0] * len(s1) ;
    hash_s2 = [0] * len(s2) ; 
 
    # Traverse through the first string 
    for i in range(len1) : 
 
        # Check if there is any matches 
        for j in range( max(0, i - max_dist), 
                    min(len2, i + max_dist + 1)) : 
             
            # If there is a match 
            if (s1[i] == s2[j] and hash_s2[j] == 0) : 
                hash_s1[i] = 1; 
                hash_s2[j] = 1; 
                match += 1; 
                break; 
         
    # If there is no match 
    if (match == 0) :
        return 0.0; 
 
    # Number of transpositions 
    t = 0; 
 
    point = 0; 
 
    # Count number of occurrences 
    # where two characters match but 
    # there is a third matched character 
    # in between the indices 
    for i in range(len1) : 
        if (hash_s1[i]) :
 
            # Find the next matched character 
            # in second string 
            while (hash_s2[point] == 0) :
                point += 1; 
 
            if (s1[i] != s2[point]) :
                point += 1;
                t += 1;
            else :
                point += 1;
                 
        t /= 2; 
 
    # Return the Jaro Similarity 
    return ((match / len1 + match / len2 +
            (match - t) / match ) / 3.0); 

def jaro_Winkler(s1, s2) : 
 
    jaro_dist = jaro_distance(s1, s2); 
 
    # If the jaro Similarity is above a threshold 
    if (jaro_dist > 0.7) :
 
        # Find the length of common prefix 
        prefix = 0; 
 
        for i in range(min(len(s1), len(s2))) :
         
            # If the characters match 
            if (s1[i] == s2[i]) :
                prefix += 1; 
 
            # Else break 
            else :
                break; 
 
        # Maximum of 4 characters are allowed in prefix 
        prefix = min(4, prefix); 
 
        # Calculate jaro winkler Similarity 
        jaro_dist += 0.1 * prefix * (1 - jaro_dist); 
 
    return jaro_dist; 

def calitateaProcesuluiDeRecunoastereATextuluiCuJaroWinklerDistance(imagePath, normalWords, normalCharacters):
    """ 
    Masoara calitatea procesului de recunoastere a cuvintelor, respectiv caracterelor folosind distanta Jaro-Winkler
    normalWords - cuvintele din imagine
    normalCharacters - literele din imagine
    """
    recognisedWords = returnAllWordsThatAppearInImage(imagePath)
    recognisedCharacters = returnAllCharactersThatAppearInAListOfWords(recognisedWords)

    calitateaCuvintelor = jaro_Winkler(recognisedWords, normalWords)
    calitateaCaracterelor = jaro_Winkler(recognisedCharacters, normalCharacters)

    return calitateaCuvintelor, calitateaCaracterelor

In [3]:
from PIL import ImageFilter, ImageEnhance, ImageOps

#PROBLEMA 3
#IMBUNATATIRI DE RECUNOASTERE A TEXTULUI

def sharpImage(image):
    """ 
    return sharpened image
    """
    sharp_img = image.filter(filter= ImageFilter.SHARPEN)
    return sharp_img


def monochromeImage(image):
    """ 
    return black and white image with high contrast
    """
    image = ImageOps.grayscale(image)
    enhancer = ImageEnhance.Contrast(image)
    contrast_factor = 2
    image = enhancer.enhance(contrast_factor)
    return image



In [4]:
import cv2 
import pytesseract 

def returnWordsFromImageUsingTesseract(imagePath):
    """  
    returns all words from an image using pytesseract library
    """
    # Load image
    img = cv2.imread(imagePath)
    size = 450,400
    img = cv2.resize(img,size)
    # Convert image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Apply threshold to convert to binary image
    threshold_img = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    # Pass the image through pytesseract
    text = pytesseract.image_to_string(threshold_img, config="--psm 6")

    print(text)

In [6]:
def main():
    # words = ["Succes", "în", "rezolvarea", "tEMELOR", "la", "LABORAtoarele", "de", "Inteligență", "Artificală!"]
    # results = calitateaProcesuluiDeRecunoastereATextuluiCuDistantaLevenshtein("test2.jpeg", words, returnAllCharactersThatAppearInAListOfWords(words))
    # print("Cuvinte corecte folosind distanta Levenshtein = ", results[0])
    # print("Caractere corecte folosind distanta Levenshtein = ", results[1])

    print("EXERCITIUL 1")
    print()

    imagePath = "test1.png"
    words = ["Google", "Cloud", "Platform"]
    characters = returnAllCharactersThatAppearInAListOfWords(words)

    resultsLevenshtein = calitateaProcesuluiDeRecunoastereATextuluiCuDistantaLevenshtein2(imagePath, words,characters)
    print("Cuvinte corecte folosind distanta Levenshtein: {} din {}".format( len(words) - resultsLevenshtein[0], len(words)))
    print("Caractere corecte folosind distanta Levenshtein: {} din {}".format( len(characters) - resultsLevenshtein[1], len(characters)))
    print()

    resultsHamming = calitateaProcesuluiDeRecunoastereATextuluiCuDistantaHamming2(imagePath, words, characters)
    print("Cuvinte corecte folosind distanta Hamming: {} din {}".format( len(words) - resultsHamming[0], len(words)))
    print("Caractere corecte folosind distanta Hamming: {} din {}".format( len(characters) - resultsHamming[1], len(characters)))
    print()

    resultsLongestCommonSubsequence = calitateaProcesuluiDeRecunoastereATextuluiCuLongestCommonSubsequence(imagePath, words, characters)
    print("Cuvinte corecte folosind LCS: {} din {}".format(resultsLongestCommonSubsequence[0], len(words)))
    print("Caractere corecte folosind LCS: {} din {}".format(resultsLongestCommonSubsequence[1], len(characters)))
    print()
    
    resultJaroWinklerDistance = calitateaProcesuluiDeRecunoastereATextuluiCuJaroWinklerDistance(imagePath, words, characters)
    print("Procentaj cuvinte corecte folosind Jaro-Winkler: {}".format(resultJaroWinklerDistance[0]))
    print("Procentaj caractere corecte folosind Jaro-Winkler: {}".format(resultJaroWinklerDistance[1]))
    print()

    print("PROBLEMA3")
    img_path = "test2.jpeg"
    image = Image.open(img_path)
    sharpened_img = sharpImage(image)
    monochrome_img = monochromeImage(sharpened_img)
    # monochrome_img.show()
    monochrome_img.save("test2Imbunatatit.jpeg")

    words = returnAllWordsThatAppearInImage("test2.jpeg")
    print("Cuvinte generate in varianta 1: ")
    print(words)

    betterWords = returnAllWordsThatAppearInImage("test2Imbunatatit.jpeg")
    print("Cuvinte generate in varianta modificarii imaginii: ")
    print(betterWords)

    print("Cuvinte generate in varianta folosirii tesseract: ")
    returnWordsFromImageUsingTesseract("test2.jpeg")

main()

EXERCITIUL 1

Cuvinte corecte folosind distanta Levenshtein: 3 din 3
Caractere corecte folosind distanta Levenshtein: 19 din 19

Cuvinte corecte folosind distanta Hamming: 3.0 din 3
Caractere corecte folosind distanta Hamming: 19.0 din 19

Cuvinte corecte folosind LCS: 3 din 3
Caractere corecte folosind LCS: 19 din 19

Procentaj cuvinte corecte folosind Jaro-Winkler: 1.0
Procentaj caractere corecte folosind Jaro-Winkler: 1.0

PROBLEMA3
Cuvinte generate in varianta 1: 
['Lucces', 'in', 'resolvarea', 'TEMELOR', 'la', 'LABORA', 'toarele', 'de', 'Inteligenta', 'Artificialà!']
Cuvinte generate in varianta modificarii imaginii: 
['Lucces', 'in', 'resolvarea', 'TEMELOR', 'la', 'LABORA', 'toarele', 'de', 'Inteligenta', 'Artificialà!']
Cuvinte generate in varianta folosirii tesseract: 
Succes in Ae Beltran
CEMELOR. La.
4AEPKA toarele "ole
Inti bg tLe call L
aa Bi
pe aa

